Task1_0829. 가상 데이터 생성 (generate_data 함수) 후 모델링 및 평가하세요
- generate_data 함수는 1000개의 랜덤 시퀀스 데이터를 생성합니다.
  - vocab_size: 시퀀스에 사용할 어휘의 크기를 설정합니다. 여기서는 100개의 단어를 사용합니다.
  - data: 각 시퀀스는 seq_length=10으로 설정된 10개의 정수(단어 인덱스)로 구성됩니다.
  - labels: 각 시퀀스에 대해 0 또는 1의 이진 레이블을 무작위로 할당합니다.

- LSTM 기반 분류 모델을 정의하고, 가상 데이터로 학습 및 검증을 수행합니다.
- 조기 종료를 통해 학습 중 성능이 더 이상 개선되지 않을 때 학습을 중단합니다.
최종적으로 테스트 데이터로 최고 성능의 모델을 평가합니다.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split


# 가상 데이터 생성 함수
def generate_data(num_samples=1000, seq_length=10):
    vocab_size = 100  # 어휘집 크기
    data = torch.randint(0, vocab_size, (num_samples, seq_length))
    labels = torch.randint(0, 2, (num_samples,))  # 0 또는 1의 레이블
    return data, labels


data, labels = generate_data()
print(data.shape, labels.shape)
for i in range(10):
    print(data[i], labels[i])

torch.Size([1000, 10]) torch.Size([1000])
tensor([98, 55, 94,  6, 24, 79, 51, 55, 43, 74]) tensor(1)
tensor([ 4, 67, 31, 79, 85, 26, 40, 31, 47, 16]) tensor(1)
tensor([48, 75, 26, 54, 70, 36, 90, 51, 68, 23]) tensor(1)
tensor([96, 41, 91, 89, 29, 88, 44, 64,  4, 94]) tensor(1)
tensor([42, 99, 29, 43, 63, 64, 53, 63,  4, 53]) tensor(0)
tensor([76, 95, 88, 31, 44, 46, 64, 53, 82, 35]) tensor(0)
tensor([85, 44, 86, 11, 99, 39, 59, 66, 46, 58]) tensor(1)
tensor([12, 52, 64, 46, 81, 16, 13,  9, 16, 62]) tensor(0)
tensor([80, 72, 93, 95, 65, 58, 65, 86, 92, 43]) tensor(1)
tensor([58, 59, 10, 45, 75, 84,  9, 38, 26,  4]) tensor(0)


In [2]:
# concatenate data and labels
dataset = TensorDataset(data, labels)
dataset_size = len(dataset)
dataset_size

1000

In [15]:
type(dataset)

torch.utils.data.dataset.TensorDataset

In [23]:
train_dataset, val_dataset, test_dataset = random_split(dataset, [700, 200, 100])

trainloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valloader = DataLoader(val_dataset, batch_size=32, shuffle=False)
testloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [24]:
# lstm model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class LSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers, num_classes):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(device)

        out = self.embedding(x)
        out, _ = self.lstm(out, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

In [25]:
model = LSTM(vocab_size=100, embedding_dim=32, hidden_dim=100, num_layers=2, num_classes=2).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# train
best_val_loss = float("inf")  # 줄어드는지 확인하기 위해 가장 큰값으로 셋팅
patience, trials = 5, 0
num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for input, labels in trainloader:
        optimizer.zero_grad()
        outputs = model(input)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # validation
    val_loss = 0.0
    model.eval()
    with torch.no_grad():
        for input, labels in valloader:
            outputs = model(input)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

    val_loss /= len(valloader)
    print(
        f"Epoch:{epoch+1}, Training Loss: {running_loss/len(trainloader)}, Validation Loss: {val_loss}"
    )

    # 조기 종료를 포함한 모델 학습 및 best model 저장
    if val_loss < best_val_loss:
        print(
            f"validation loss decreased from {best_val_loss:.6f} to {val_loss:.6f}. Saving the model..."
        )
        best_val_loss = val_loss
        trials = 0
        torch.save(model.state_dict(), "best_model_fashion.pth")
    else:
        trials += 1
        if trials >= patience:
            print(f"early stop on epoch {epoch+1}")
            break

# load weight
model.load_state_dict(torch.load("best_model_fashion.pth", weights_only=True))

# evaluation on testloader
correct, total = 0, 0
model.eval()
with torch.no_grad():
    for input, labels in testloader:
        outputs = model(input)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy: {100 * correct / total}%")

Epoch:1, Training Loss: 0.6948290196332064, Validation Loss: 0.6947515521730695
validation loss decreased from inf to 0.694752. Saving the model...
Epoch:2, Training Loss: 0.6939650638536974, Validation Loss: 0.6948898094041007
Epoch:3, Training Loss: 0.6935347589579496, Validation Loss: 0.6947405338287354
validation loss decreased from 0.694752 to 0.694741. Saving the model...
Epoch:4, Training Loss: 0.6931178542700681, Validation Loss: 0.694702787058694
validation loss decreased from 0.694741 to 0.694703. Saving the model...
Epoch:5, Training Loss: 0.6932640427892859, Validation Loss: 0.6948157548904419
Epoch:6, Training Loss: 0.6933223496783864, Validation Loss: 0.6946726271084377
validation loss decreased from 0.694703 to 0.694673. Saving the model...
Epoch:7, Training Loss: 0.6929137815128673, Validation Loss: 0.6946959836142403
Epoch:8, Training Loss: 0.6935121010650288, Validation Loss: 0.6946798392704555
Epoch:9, Training Loss: 0.6931159008633007, Validation Loss: 0.69531724282

# ex


텍스트를 형태소로 분리해야하니 토큰나이저를 사용해야한다.

토크나이제이션을 직접 할 일은 없지만 원리를 알면 잘 쓸수 있다.

단어는 희소행렬이다 그래서 인베딩을 해야한다.

자연어 처리니 lstm을 쓴다.



#### nn.Embedding(vocag_size,embed_dim)
- 텍스트 데이터(LSTM 모델): 텍스트 데이터는 이산적인 정수로 표현되므로, 이 정수들을 고차원 벡터로 매핑하는 nn.Embedding 계층이 필요합니다. 이 임베딩 계층은 단어 간의 의미적 유사성을 학습하는 데 유용합니다.

- 이미지 데이터(CNN 모델): 이미지 데이터는 이미 공간적 구조를 가진 연속적인 값(픽셀)으로 표현되므로, 임베딩 계층이 필요하지 않습니다. 대신, 합성곱 계층이 이미지의 패턴을 학습하는 데 사용됩니다. 캡쳐, 풀링 계층을 통해 이미지의 공간적 구조를 유지하면서 차원을 줄이거나 특징을 추출할 수 있습니다.

LSTM 모델의 경우 텍스트 데이터의 정수 인덱스를 벡터로 변환하기 위해 nn.Embedding- 토치용 임베딩 이 필요하지만, CNN 모델에서는 이미지 데이터를 처리하는 데 이미 직접적인 합성곱 연산이 사용되므로 임베딩 계층이 필요하지 않습니다.

nn.BCEWithLogitsLoss는 PyTorch에서 이진 분류(Binary Classification) 작업을 수행할 때 주로 사용하는 손실 함수입니다. 이 함수는 이진 교차 엔트로피 손실(Binary Cross Entropy Loss)와 시그모이드(Sigmoid) 함수를 결합한 형태로 제공됩니다.

로짓(Logit) 값은 모델의 출력값을 의미하며, BCEWithLogitsLoss 함수는 이 출력값에 시그모이드 함수를 적용하여 0과 1 사이의 확률값으로 변환한 뒤, 이진 교차 엔트로피 손실을 계산합니다.

버트 : 긍부정을 판단하는데 사용된다.

개체명 인식 : 개체명을 인식하는데 사용된다.

앞의 문맥을 보고 다음 단어가 맞는지 틀리는지 판단.

0,1 은 각 학습에 대한 맞고 틀리고 나타내는 것이다.

단순 토큰나이제이션(단어로 나누기만 하는 것)을 하면 안쓰는 단어면 문제가 생김

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split


# 가상 데이터 생성 함수
def generate_data(num_samples=1000, seq_length=10):
    vocab_size = 100  # 어휘집 크기
    data = torch.randint(0, vocab_size, (num_samples, seq_length))
    labels = torch.randint(0, 2, (num_samples,))  # 0 또는 1의 레이블
    return data, labels


data, labels = generate_data()

# 텐서 데이터셋 및 데이터 로더 생성
dataset = TensorDataset(data, labels)
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - (train_size + val_size)
train_dataset, val_dataset, test_dataset = random_split(
    dataset, [train_size, val_size, test_size]
)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


# LSTM 모델 클래스 정의
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, output_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, text):
        embedded = self.embedding(text)
        lstm_out, (hidden, _) = self.lstm(embedded)
        hidden = hidden[-1, :, :]
        return self.fc(hidden)


model = LSTMModel(vocab_size=100, embed_dim=50, hidden_dim=100, output_dim=1)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters())


# 훈련 함수
def train(model, train_loader, optimizer, criterion):
    model.train()
    total_loss = 0
    for data, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(data).squeeze(1)
        loss = criterion(outputs, labels.float())
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)


# 평가 함수
def evaluate(model, data_loader, criterion):
    model.eval()
    total_loss = 0
    total_accuracy = 0
    with torch.no_grad():
        for data, labels in data_loader:
            outputs = model(data).squeeze(1)
            loss = criterion(outputs, labels.float())
            total_loss += loss.item()
            predictions = torch.round(torch.sigmoid(outputs))
            correct_predictions = (predictions == labels.unsqueeze(1)).float()
            total_accuracy += correct_predictions.sum().item()
    return total_loss / len(data_loader), total_accuracy / len(data_loader.dataset)


# 조기 종료 로직을 포함한 훈련 및 검증 과정
best_val_loss = float("inf")  # float('inf')는 파이썬에서 양의 무한대를 나타내는 방식
patience = 3
trials = 0

for epoch in range(20):
    train_loss = train(model, train_loader, optimizer, criterion)
    val_loss, val_accuracy = evaluate(model, val_loader, criterion)
    print(
        f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}"
    )

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        trials = 0
        torch.save(model.state_dict(), "best_model.pth")
    else:
        trials += 1
        if trials >= patience:
            print("조기 종료 발생")
            break

# 테스트 데이터로 최고 모델 평가
model.load_state_dict(torch.load("best_model.pth", weights_only=True))
test_loss, test_accuracy = evaluate(model, test_loader, criterion)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

Epoch 1, Train Loss: 0.6938, Val Loss: 0.6976, Val Accuracy: 14.9467
Epoch 2, Train Loss: 0.6777, Val Loss: 0.7003, Val Accuracy: 15.0000
Epoch 3, Train Loss: 0.6625, Val Loss: 0.7085, Val Accuracy: 14.9867
Epoch 4, Train Loss: 0.6384, Val Loss: 0.7197, Val Accuracy: 15.2000
조기 종료 발생
Test Loss: 0.6979, Test Accuracy: 14.9867
